In [2]:
import pandas as pd
import pyodbc
import numpy as np
import sqlalchemy as db
import csv
import datetime

In [166]:
# Connect to SQL Server
def getConnection():
    conn = pyodbc.connect(Driver='{SQL Server Native Client 11.0}'
                      ,host='APAUACTPAP161'
                      , Database='DW_Actuarial'
                      , Trusted_Connection= 'yes'
                     )
    return conn
	
conn=getConnection()


Source_Server = 'APAUACTPAP161'
Source_Database = 'DW_Actuarial'

conn_str = (
            "Driver={SQL Server Native Client 11.0};"
            f"Server={Source_Server};"
            f"Database={Source_Database};"
            "Trusted_Connection=yes;"
           )

engine_SRC = db.create_engine(f'mssql+pyodbc:///?odbc_connect={conn_str}',fast_executemany=True)

In [167]:
query1='''Select TABLE_SCHEMA,TABLE_NAME from INFORMATION_SCHEMA.TABLES
Where TABLE_SCHEMA in ('ENTSIA','EPLEXTRACT','GenNext','GenNext2','KAIZEN','PCW','EVOLUTION')
Order by TABLE_SCHEMA'''
conn=getConnection()
TableList=pd.read_sql(query1,conn)

TableList=TableList.reset_index(drop=True)
TableList.head(2)


In [168]:
# Create an empty DataFrame
final_data = pd.DataFrame(columns=['Schema_Name', 'Table_Name', 'Last_Update'])
final_data

In [169]:
for index,row in TableList.iterrows():
     schema=row[0]
     table_name=row[1]
     query = f''' 
               SELECT TOP(1) '{schema}' as Schema_Name ,
               '{table_name}' as Table_Name, 
               CopyDate as Last_Update
               from {schema}.{table_name}
               ''' 
     ##print(index,row)
     ##if index> 10:
     ##     break
     try:             
         RawData = pd.read_sql(query, conn)
         ##print(RawData)
         final_data = pd.concat([final_data, RawData], ignore_index=True)
         ##print(new_df)
         
     except Exception as e:
          continue

In [170]:
##final_data.to_csv(r'C:\Users\SASUMAN\Desktop\Weekly_DB_Chnage_Report\finaldata.csv', index=True)

In [171]:
grouped_data = final_data.groupby('Schema_Name').agg({'Last_Update': ['min', 'max']})
grouped_data.columns = ['Min_Last_Update', 'Max_Last_Update']
grouped_data

In [172]:
from pandasql import sqldf
from datetime import datetime
mysql = lambda q: sqldf(q, globals())
query1=mysql("select Schema_name,Max_Last_Update as Last_Updated from grouped_data where Schema_name ='PCW'")
query2=mysql("select Schema_name,Min_Last_Update as Last_Updated from grouped_data where Schema_name !='PCW'")
print(query1)
print(query2)

In [173]:
Final_Output = pd.concat([query1, query2], axis=0)

# Convert 'Last_Updated' column to datetime
Final_Output['Last_Updated'] = pd.to_datetime(Final_Output['Last_Updated'])

# Add 30 days to 'Last_Updated' column
Final_Output['Next_Update'] = Final_Output['Last_Updated'] + pd.DateOffset(days=30)

# Adjust next update date if it falls on a weekend
for i, value in enumerate(Final_Output['Next_Update']):
    while value.weekday() > 4:  # 4 represents Friday, weekdays are numbered 0 (Monday) to 6 (Sunday)
        value += pd.DateOffset(days=1)
    Final_Output['Next_Update'].iat[i] = value
    
##Final_Output["Frequency"]="Monthly"

current_date = datetime.now()
# Convert current_date to datetime data type
current_date = pd.to_datetime(current_date)

Final_Output['Execution_Status'] = ['COMPLETED' if x >= current_date else 'PENDING' for x in Final_Output['Next_Update']]
Final_Output.loc[pd.isnull(Final_Output['Next_Update']), 'Execution_Status'] = 'TBA'
Final_Output.loc[pd.isnull(Final_Output['Last_Updated']), 'Execution_Status'] = 'TBA'

Final_Output["COMMENTS"]='Latest Data till X is Updated in Prod Environment'

# Update 'COMMENTS' column with month and year from 'Last_Updated' column - 1 month
Final_Output['COMMENTS'] = Final_Output.apply(lambda row: row['COMMENTS'].replace("X",
                                                              (row['Last_Updated'] - pd.DateOffset(months=1)).strftime('%B %Y'))
                                          if not pd.isnull(row['Last_Updated']) else row['COMMENTS'],
                          axis=1)


def label_diff(date_diff):
    if date_diff.days >= 30:
        return 'Monthly'
    else:
        return 'Weekly'

Final_Output['Frequency'] = Final_Output.apply(lambda x: label_diff(x['Next_Update'] - x['Last_Updated']), axis=1)







# IF we have still this "Latest Data till X is Updated in Prod Environment" in comment then replace it with 
# "There will be some issue we will check and update accordingly"
Final_Output['COMMENTS'] = Final_Output['COMMENTS'].str.replace("Latest Data till X is Updated in Prod Environment", "There will be some issue we will check and update accordingly")


Final_Output['Last_Updated'] = Final_Output['Last_Updated'].dt.strftime('%d-%B-%Y')
Final_Output['Next_Update'] = Final_Output['Next_Update'].dt.strftime('%d-%B-%Y')
Final_Output=Final_Output.sort_values(by=['Last_Updated'], ascending=True).reset_index(drop=True)


Final_Output=Final_Output.reset_index(drop=True)
Final_Output=Final_Output[["Schema_Name","Frequency" ,"Last_Updated", "Next_Update","Execution_Status","COMMENTS"]]
Final_Output

In [174]:
##Final_Output.to_csv(r'C:\Users\SASUMAN\Desktop\Weekly_DB_Chnage_Report\DB_Change_Report_30th_Apr.csv', index=False)

In [4]:
import pandas as pd

data = [
    ['KAIZEN','ENTSIA','EPLEXTRACT','GenNext','GenNext2','EVOLUTION','PCW'],
    ['Monthly','Monthly','Monthly','Monthly','Monthly','Monthly','Monthly'],
    ['03-July-2024','05-June-2024','06-June-2024','13-June-2024','13-June-2024','14-June-2024','27-June-2024'],
    ['02-August-2024','05-July-2024','08-July-2024','15-July-2024','15-July-2024','15-July-2024','29-July-2024'],
    ['COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED','COMPLETED'],
    ['Latest Data till June 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment',
     'Latest Data till May 2024 is Updated in Prod Environment']
]

columns = ['Schema_Name', 'Frequency', 'Last_Updated', 'Next_Update', 'Execution_Status', 'COMMENTS']

# Transpose the data so it aligns with the columns
data_transposed = list(map(list, zip(*data)))

Final_Output = pd.DataFrame(data_transposed, columns=columns)

print(Final_Output)




  Schema_Name Frequency  Last_Updated     Next_Update Execution_Status  \
0      KAIZEN   Monthly  03-July-2024  02-August-2024        COMPLETED   
1      ENTSIA   Monthly  05-June-2024    05-July-2024        COMPLETED   
2  EPLEXTRACT   Monthly  06-June-2024    08-July-2024        COMPLETED   
3     GenNext   Monthly  13-June-2024    15-July-2024        COMPLETED   
4    GenNext2   Monthly  13-June-2024    15-July-2024        COMPLETED   
5   EVOLUTION   Monthly  14-June-2024    15-July-2024        COMPLETED   
6         PCW   Monthly  27-June-2024    29-July-2024        COMPLETED   

                                            COMMENTS  
0  Latest Data till June 2024 is Updated in Prod ...  
1  Latest Data till May 2024 is Updated in Prod E...  
2  Latest Data till May 2024 is Updated in Prod E...  
3  Latest Data till May 2024 is Updated in Prod E...  
4  Latest Data till May 2024 is Updated in Prod E...  
5  Latest Data till May 2024 is Updated in Prod E...  
6  Latest Data till Ma

In [7]:
import datetime
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook

# Load the existing Excel file
filename = "Weekly_SSIS_Status_Report.xlsx"
book = load_workbook(filename)

# Select the worksheet within the Excel file
worksheet = book['Report_Snapshot']

# Determine the starting row and column indices
# start_row = 185
# start_col = 1

last_row = len(worksheet["D"])  # Get the last row in column D
start_row = last_row + 4  # Add 5 more rows
start_col = 1

print(f'last_row index :{last_row}')
print(f'start_row index :{start_row}')
print(f'start_col index :{start_col}')

# Set header formatting (blue font color and cell fill color)
header_font = Font(color="0000FF", bold=True)
header_fill = PatternFill(start_color="A9D0F5", end_color="A9D0F5", fill_type="solid")

# Add header "Run_Date" in cell (start_row, start_col)
run_date = datetime.date.today().strftime('%d-%B-%Y')
header_cell = worksheet.cell(row=start_row, column=start_col, value="Run_Date")
header_cell.font = header_font
header_cell.fill = header_fill
header_cell.alignment = Alignment(horizontal='center')

# Add the current date in the corresponding cell (start_row+1, start_col)
run_date_cell = worksheet.cell(row=start_row+1, column=start_col, value=run_date)
run_date_cell.alignment = Alignment(horizontal='center')

# Save the changes to the Excel file
book.save(filename)

last_row index :218
start_row index :222
start_col index :1


In [11]:
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment
import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd  # Make sure to import pandas

# To get the Last month from the current date
current_date = datetime.datetime.now()

# Calculate previous month
previous_month = current_date - relativedelta(months=1)
previous_month_name = previous_month.strftime("%B %Y")

# Create a DataFrame with previous month name and year
month_of_upload_data = pd.DataFrame(data=[previous_month_name])

month_of_upload_data_str = month_of_upload_data.to_string(index=False, header=False).strip()

# Load the workbook
book = openpyxl.load_workbook("Weekly_SSIS_Status_Report.xlsx")

# Select the worksheet within the Excel file
worksheet = book["Report_Snapshot"]

# Determine the starting row and column indices
# Assuming start_row and start_col are defined earlier in your code
# If not, you need to define these variables

# If start_row and start_col are not defined, you need to calculate them
# Example calculation (this should be replaced with your actual logic)
start_row = len(worksheet["D"]) + 4  # Adjust as needed
start_col = 4

print(start_row)
print(start_col)

# Set header formatting (blue font color and cell fill color)
header_font = Font(color="0000FF", bold=True)
header_fill = PatternFill(start_color="A9D0F5", end_color="A9D0F5", fill_type="solid")

##################################
cell_range = f"{worksheet.cell(row=start_row-6, column=start_col-1).coordinate}:{worksheet.cell(row=start_row + 15, column=start_col + 8).coordinate}"

# Define the color to fill
fill_color = "F6E8E6"  # Red color

# Create a PatternFill object with the desired color
fill = PatternFill(start_color=fill_color, fill_type="solid")

# Iterate through each cell in the range and apply the fill
for row in worksheet[cell_range]:
    for cell in row:
        cell.fill = fill
########################################
# Add "SOME TEXT" at cell -5 from start row
text1 = f'{month_of_upload_data_str} Data Upload Status'
cell1 = worksheet.cell(row=start_row - 5, column=start_col, value=text1)

# Set font style
font = Font(bold=True, size=15)
cell1.font = font

# Set fill color
fill = PatternFill(fgColor="00FF00")  # Green color fill 05FB51
cell1.fill = fill

# Add "some text2" at cell -4 from start row

# Define Source_Server and Source_Database
Source_Server = 'APAUACTPAP161'
Source_Database = 'DW_Actuarial'

text2 = "Server Details :"
cell2 = worksheet.cell(row=start_row - 4, column=start_col, value=text2)

text3 = f'{Source_Server}'
cell3 = worksheet.cell(row=start_row - 4, column=start_col + 1, value=text3)

# Add "some text3" at cell -3 from start row
text4 = "Database Name :"
cell4 = worksheet.cell(row=start_row - 3, column=start_col, value=text4)

text5 = f'{Source_Database}'
cell5 = worksheet.cell(row=start_row - 3, column=start_col + 1, value=text5)

# Save the changes to the Excel file
book.save("Weekly_SSIS_Status_Report.xlsx")


227
4


In [13]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.worksheet.table import Table, TableStyleInfo

# Load the existing Excel file
filename = "Weekly_SSIS_Status_Report.xlsx"
book = load_workbook(filename)

# Select the worksheet within the Excel file
worksheet = book['Report_Snapshot']

# Define the DataFrame to be added (make sure Final_Output is defined before this)
# For example, replace the following line with your actual DataFrame definition
Final_Output = pd.DataFrame({
    'Schema_Name': ['KAIZEN', 'ENTSIA', 'EPLEXTRACT', 'GenNext', 'GenNext2', 'EVOLUTION', 'PCW'],
    'Frequency': ['Monthly'] * 7,
    'Last_Updated': ['03-July-2024', '05-June-2024', '06-June-2024', '13-June-2024', '13-June-2024', '14-June-2024', '27-June-2024'],
    'Next_Update': ['02-August-2024', '05-July-2024', '08-July-2024', '15-July-2024', '15-July-2024', '15-July-2024', '29-July-2024'],
    'Execution_Status': ['COMPLETED'] * 7,
    'COMMENTS': [
        'Latest Data till June 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment',
        'Latest Data till May 2024 is Updated in Prod Environment'
    ]
})

# Convert the DataFrame to rows
rows = list(dataframe_to_rows(Final_Output, index=False, header=True))

# Determine the starting row and column indices
start_row = len(worksheet['D']) + 3  # Adjust as needed
start_col = 4

print(start_row)
print(start_col)

# Set header formatting (blue font color and cell fill color)
header_font = Font(color="0000FF", bold=True)
header_fill = PatternFill(start_color="A9D0F5", end_color="A9D0F5", fill_type="solid")
for col_index, header_value in enumerate(rows[0], start=start_col):
    header_cell = worksheet.cell(row=start_row, column=col_index, value=header_value)
    header_cell.font = header_font
    header_cell.fill = header_fill
    header_cell.alignment = Alignment(horizontal='center')

# Insert the rows in the worksheet
for row_index, row_data in enumerate(rows[1:], start=start_row + 1):
    for col_index, cell_value in enumerate(row_data, start=start_col):
        cell = worksheet.cell(row=row_index, column=col_index, value=cell_value)

# Determine the range for the table (including the header)
num_rows = len(rows) - 1
num_columns = len(rows[0])
end_row = start_row + num_rows
end_col = start_col + num_columns - 1
data_range = f"{worksheet.cell(row=start_row, column=start_col).coordinate}:{worksheet.cell(row=end_row, column=end_col).coordinate}"

# Create table for the data range (including the header)
table_name = f"Table{len(worksheet.tables) + 1}"
table = Table(displayName=table_name, ref=data_range)

# Apply table style
table_style = TableStyleInfo(name="TableStyleLight10", showFirstColumn=False, showLastColumn=False, showRowStripes=True)
table.tableStyleInfo = table_style

# Add table to the worksheet
worksheet.add_table(table)

# Save the changes to the Excel file
book.save(filename)


245
4


In [178]:
################################################   END OF THE CODE ##########################################################